In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import numpy as np
import datetime

pd.options.display.precision = 4

In [2]:
lines = []

with open(r"C:\Users\charl\Desktop\VS_Code\2024\STREAMLIT_APP\POZOS_COMPILADO.csv", "r") as fp:
    lines = fp.readlines()[10:]

lines_2 = []
for line in lines:
    lines_2.append(line[:-1].split(","))

In [3]:
df = pd.DataFrame(lines_2)
df.head(3)

,0,1,2,3,4,5,6,7,8,9,10
0,Fecha,Identificador,Nombre_del_pozo,Cuenca,Asignación_o_Contrato,Hidrocarburos_Liquidos_(Mbd),Petróleo_(Mbd),Condensado_(Mbd),Gas_asociado_(MMpcd),Gas_no_asociado_(MMpcd),Agua_(Mbd)
1,01-06-1930,1000023609,Poza Rica-2,TAMPICO-MISANTLA,,0.009015,0.009015,0.0,0.00418,0.0,0.0
2,01-07-1930,1000023609,Poza Rica-2,TAMPICO-MISANTLA,,0.020695,0.020695,0.0,0.009594,0.0,0.0


In [4]:
df = pd.DataFrame(lines_2)
df.columns = ["Fecha", "id", "Pozo", "Cuenca", "Contrato", "Ql_(Mbd)", "Qo_(Mbd)", "Cond_(Mbd)", 
             "Qga_(MMpcd)", "Qgna_(MMpcd)", "Qw_(Mbd)"]
df = df.drop(df.index[0])
df = df.reset_index(drop=True)
df["Fecha"] = pd.to_datetime(df["Fecha"], dayfirst=True)
df.iloc[:, 5:] = df.iloc[:, 5:].astype(float)
df.shape

C:\Users\charl\AppData\Local\Temp\ipykernel_20680\4275472138.py:7: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df.iloc[:, 5:] = df.iloc[:, 5:].astype(float)


(3254852, 11)

In [5]:
len(df["Pozo"].unique())

33895

In [6]:
len(df)

3254852

In [12]:

tablero = pd.DataFrame()
pozos = list(df.Pozo.unique())
cont = 0
tot = len(pozos)
fecha_reporte = datetime.datetime(2024, 3, 1)

for pozo in pozos:
    # Creating a copy
    df_pozo = df[df["Pozo"] == pozo].copy()
    # df_pozo.drop_duplicates("Fecha", inplace=True)

    # Merge dataframes to fill the date gaps
    if fecha_reporte.year - df_pozo["Fecha"].iloc[-1].year > 10:
        d = pd.date_range(df_pozo["Fecha"].iloc[0], df_pozo["Fecha"].iloc[-1] + pd.DateOffset(years=1), freq='MS')
    else:
        d = pd.date_range(df_pozo["Fecha"].iloc[0], fecha_reporte, freq='MS')
    
    d_index = pd.DataFrame({"Fecha": d})
    df_pfinal = pd.merge(df_pozo, d_index, on="Fecha", how="right")
    df_pfinal.iloc[:, 5:] = df_pfinal.iloc[:, 5:].fillna(0)
    df_pfinal.iloc[:, 1:5] = df_pfinal.iloc[:, 1:5].fillna(method="ffill")

    # Create the Cumulative columns
    cols = ["Qo_(Mbd)", "Cond_(Mbd)", "Qga_(MMpcd)", "Qgna_(MMpcd)", "Qw_(Mbd)"]
    new_cols = ["Qo_ac(Mbd)", "Cond_ac(Mbd)", "Qga_ac(MMpcd)", "Qgna_ac(MMpcd)", "Qw_ac(Mbd)"]

    for new_col, col in zip(new_cols, cols):
        df_pfinal[new_col] = df_pfinal[col].cumsum()

    tablero = pd.concat([tablero, df_pfinal], ignore_index=True)
    cont += 1
    perc = (cont/tot)*100
    print(f"Pozo {cont} de {tot} : {perc:.2f} %")
# tablero["Pozo"].unique()

Pozo 1 de 33895 : 0.00 %
Pozo 2 de 33895 : 0.01 %
Pozo 3 de 33895 : 0.01 %
Pozo 4 de 33895 : 0.01 %
Pozo 5 de 33895 : 0.01 %
Pozo 6 de 33895 : 0.02 %
Pozo 7 de 33895 : 0.02 %
Pozo 8 de 33895 : 0.02 %
Pozo 9 de 33895 : 0.03 %
Pozo 10 de 33895 : 0.03 %
Pozo 11 de 33895 : 0.03 %
Pozo 12 de 33895 : 0.04 %
Pozo 13 de 33895 : 0.04 %
Pozo 14 de 33895 : 0.04 %
Pozo 15 de 33895 : 0.04 %
Pozo 16 de 33895 : 0.05 %
Pozo 17 de 33895 : 0.05 %
Pozo 18 de 33895 : 0.05 %
Pozo 19 de 33895 : 0.06 %
Pozo 20 de 33895 : 0.06 %
Pozo 21 de 33895 : 0.06 %
Pozo 22 de 33895 : 0.06 %
Pozo 23 de 33895 : 0.07 %
Pozo 24 de 33895 : 0.07 %
Pozo 25 de 33895 : 0.07 %
Pozo 26 de 33895 : 0.08 %
Pozo 27 de 33895 : 0.08 %
Pozo 28 de 33895 : 0.08 %
Pozo 29 de 33895 : 0.09 %
Pozo 30 de 33895 : 0.09 %
Pozo 31 de 33895 : 0.09 %
Pozo 32 de 33895 : 0.09 %
Pozo 33 de 33895 : 0.10 %
Pozo 34 de 33895 : 0.10 %
Pozo 35 de 33895 : 0.10 %
Pozo 36 de 33895 : 0.11 %
Pozo 37 de 33895 : 0.11 %
Pozo 38 de 33895 : 0.11 %
Pozo 39 de 33895 : 0.

In [14]:
tablero.shape

(5884901, 16)

In [13]:
len(tablero)

5884901

In [15]:
tablero.to_csv("tablero.csv", index=False, float_format='%.3f', encoding="latin1")

### Plots

In [8]:
# Oil
def plot_oil(df, x, y, pozo):
    fig = px.line(df, x=x, y=y, markers=True, title=f"<b>{pozo}</b>" + f"<br>{df['Cuenca'].iloc[0]} - {df['Contrato'].iloc[0]}")
    fig.update_layout(title_x=0.5, title_y=0.9, plot_bgcolor="white")
    fig.update_xaxes(showline=False)
    fig.update_yaxes(showline=True, gridcolor="#eeeeee")
    fig.update_traces(line_color="#1F8A70")
    return fig

# Gas
def plot_gas(df, x, y, pozo):
    fig = px.line(df, x=x, y=y, markers=True, title=f"{pozo}")
    fig.update_layout(title_x=0.5, title_y=0.83)
    fig.update_traces(line_color="red")
    return fig

# Water
def plot_water(df, x, y, pozo):
    fig = px.line(df, x=x, y=y, markers=True, title=f"{pozo}")
    fig.update_layout(title_x=0.5, title_y=0.83)
    fig.update_traces(line_color="blue")
    return fig


In [41]:
pozo = "Poza Rica-2"
tablero_pozo = tablero[tablero["Pozo"] == pozo]
plot_oil(tablero_pozo, tablero_pozo["Fecha"], tablero_pozo["Qo_(Mbd)"], pozo)

### Producción de aceite

In [42]:
plot_oil(tablero_pozo, tablero_pozo["Fecha"], tablero_pozo["Qo_(Mbd)"], pozo)

### Producción acumulada aceite

In [62]:
plot_oil(tablero_pozo, tablero_pozo["Fecha"], tablero_pozo["Qo_ac(Mbd)"], pozo)

### Producción de gas

In [64]:
plot_gas(tablero_pozo, tablero_pozo["Fecha"], tablero_pozo["Qga_(MMpcd)"], pozo)

### Producción acumulada de gas

In [65]:
plot_gas(tablero_pozo, tablero_pozo["Fecha"], tablero_pozo["Qga_ac(MMpcd)"], pozo)

### Producción de agua

In [61]:
plot_water(tablero_pozo, tablero_pozo["Fecha"], tablero_pozo["Qw_(Mbd)"], pozo)

### Producción acumulada de agua

In [60]:
plot_water(tablero_pozo, tablero_pozo["Fecha"], tablero_pozo["Qw_ac(Mbd)"], pozo)

### Producción de gas no asociado

In [59]:
plot_gas(tablero_pozo, tablero_pozo["Fecha"], tablero_pozo["Qgna_(MMpcd)"], pozo)

### Producción acumulada de gas no asociado

In [58]:
plot_water(tablero_pozo, tablero_pozo["Fecha"], tablero_pozo["Qgna_ac(MMpcd)"], pozo)

### Producción de Condensado

In [57]:
plot_oil(tablero_pozo, tablero_pozo["Fecha"], tablero_pozo["Cond_(Mbd)"], pozo)

In [56]:
plot_oil(tablero_pozo, tablero_pozo["Fecha"], tablero_pozo["Cond_ac(Mbd)"], pozo)

In [ ]:
# tablero_pozo.to_csv("df_B-23.csv", index=False, float_format='%.3f')

In [1]:
import pandas as pd

tablero = pd.read_csv("tablero.csv")
tablero.shape

(8287193, 16)

In [14]:
tablero.Contrato.fillna("A-0280-3M-CAMPO POZA RICA", inplace=True)

In [19]:
len(tablero.Contrato.unique())

430

In [20]:
tablero.Contrato.isna().unique().sum()

0

In [21]:
list(tablero.Contrato.unique())[:5]

['A-0280-3M-CAMPO POZA RICA',
 'A-0303-2M - CAMPO SANTA ROSALIA',
 'A-0112-3M - CAMPO CUITLAHUAC',
 'A-0296-2M - CAMPO SAMARIA',
 'A-0113-2M - CAMPO CULEBRA']

In [32]:
df_burgos.head(3)

,Fecha,id,Pozo,Cuenca,Contrato,Ql_(Mbd),Qo_(Mbd),Cond_(Mbd),Qga_(MMpcd),Qgna_(MMpcd),Qw_(Mbd),Qo_ac(Mbd),Cond_ac(Mbd),Qga_ac(MMpcd),Qgna_ac(MMpcd),Qw_ac(Mbd)
60881,1946-04-01,1000019310,Misión-2,BURGOS,A-0280-3M-CAMPO POZA RICA,0.008,0.0,0.008,0.0,0.396,0.0,0.0,0.008,0.0,0.396,0.0
60882,1946-05-01,1000019310,Misión-2,BURGOS,A-0280-3M-CAMPO POZA RICA,0.000,0.0,0.000,0.0,0.000,0.0,0.0,0.008,0.0,0.396,0.0
60883,1946-06-01,1000019310,Misión-2,BURGOS,A-0280-3M-CAMPO POZA RICA,0.000,0.0,0.000,0.0,0.000,0.0,0.0,0.008,0.0,0.396,0.0


In [46]:
df_burgos = tablero[(tablero["Cuenca"] == "BURGOS")]
len(list(df_burgos["Pozo"].unique()))

9534

In [43]:
df_burgos[df_burgos["Contrato"] == "A-0280-3M-CAMPO POZA RICA"]

,Fecha,id,Pozo,Cuenca,Contrato,Ql_(Mbd),Qo_(Mbd),Cond_(Mbd),Qga_(MMpcd),Qgna_(MMpcd),Qw_(Mbd),Qo_ac(Mbd),Cond_ac(Mbd),Qga_ac(MMpcd),Qgna_ac(MMpcd),Qw_ac(Mbd)
60881,1946-04-01,1000019310,Misión-2,BURGOS,A-0280-3M-CAMPO POZA RICA,0.008,0.0,0.008,0.0,0.396,0.0,0.0,0.008,0.0,0.396,0.0
60882,1946-05-01,1000019310,Misión-2,BURGOS,A-0280-3M-CAMPO POZA RICA,0.000,0.0,0.000,0.0,0.000,0.0,0.0,0.008,0.0,0.396,0.0
60883,1946-06-01,1000019310,Misión-2,BURGOS,A-0280-3M-CAMPO POZA RICA,0.000,0.0,0.000,0.0,0.000,0.0,0.0,0.008,0.0,0.396,0.0
60884,1946-07-01,1000019310,Misión-2,BURGOS,A-0280-3M-CAMPO POZA RICA,0.000,0.0,0.000,0.0,0.000,0.0,0.0,0.008,0.0,0.396,0.0
60885,1946-08-01,1000019310,Misión-2,BURGOS,A-0280-3M-CAMPO POZA RICA,0.000,0.0,0.000,0.0,0.000,0.0,0.0,0.008,0.0,0.396,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7095527,2023-11-01,1000010186,Cuitláhuac-591D,BURGOS,A-0280-3M-CAMPO POZA RICA,0.000,0.0,0.000,0.0,0.000,0.0,0.0,0.000,0.0,0.106,0.0
7095528,2023-12-01,1000010186,Cuitláhuac-591D,BURGOS,A-0280-3M-CAMPO POZA RICA,0.000,0.0,0.000,0.0,0.000,0.0,0.0,0.000,0.0,0.106,0.0
7095529,2024-01-01,1000010186,Cuitláhuac-591D,BURGOS,A-0280-3M-CAMPO POZA RICA,0.000,0.0,0.000,0.0,0.000,0.0,0.0,0.000,0.0,0.106,0.0
7095530,2024-02-01,1000010186,Cuitláhuac-591D,BURGOS,A-0280-3M-CAMPO POZA RICA,0.000,0.0,0.000,0.0,0.000,0.0,0.0,0.000,0.0,0.106,0.0


In [2]:
import pandas as pd

tab = pd.read_csv("tablero_julia.csv", encoding="latin-1")
tab.head()

C:\Users\charl\AppData\Local\Temp\ipykernel_24716\3996461415.py:3: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  tab = pd.read_csv("tablero_julia.csv", encoding="latin-1")


,Fecha,id,Pozo,Cuenca,Contrato,Ql_(Mbd),Qo_(Mbd),Cond_(Mbd),Qga_(MMpcd),Qgna_(MMpcd),Qw_(Mbd),Qo_ac(Mbd),Cond_ac(Mbd),Qga_ac(MMpcd),Qgna_ac(MMpcd),Qw_ac(Mbd)
0,1930-06-01,1000023609,Poza Rica-2,TAMPICO-MISANTLA,NaN,0.009015,0.009015,0.0,0.004180,0.0,0.0,0.009015,0.0,0.004180,0.0,0.0
1,1930-07-01,1000023609,Poza Rica-2,TAMPICO-MISANTLA,NaN,0.020695,0.020695,0.0,0.009594,0.0,0.0,0.029710,0.0,0.013774,0.0,0.0
2,1930-08-01,1000023609,Poza Rica-2,TAMPICO-MISANTLA,NaN,0.020695,0.020695,0.0,0.009594,0.0,0.0,0.050405,0.0,0.023368,0.0,0.0
3,1930-09-01,1000023609,Poza Rica-2,TAMPICO-MISANTLA,NaN,0.020756,0.020756,0.0,0.009622,0.0,0.0,0.071161,0.0,0.032990,0.0,0.0
4,1930-10-01,1000023609,Poza Rica-2,TAMPICO-MISANTLA,NaN,0.020695,0.020695,0.0,0.009594,0.0,0.0,0.091856,0.0,0.042584,0.0,0.0
